## 模型融合model ensemble
1.准备数据-载入各模型的预测概率  
2.融合训练-找到各模型的权重  
3.模型融合-Random Forest,LightGBM,XGBoost_LR

In [1]:
from sklearn.tree import DecisionTreeClassifier
from lightgbm.sklearn import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn import metrics
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
import lightgbm
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
import scipy.io as sio

### 1.准备数据

In [2]:
rf_pred = pickle.load(open("./result/rf_pred.pkl","rb")).astype('float32')[:,1]
lgbm_pred = pickle.load(open("./result/lgbm_pred.pkl","rb")).astype('float32')[:,1]
xgb_lr_pred = pickle.load(open("./result/xgb_lr_pred.pkl","rb")).astype('float32')[:,1]

In [3]:
%%time
X_train = np.hstack((rf_pred.reshape(-1,1),lgbm_pred.reshape(-1,1),\
                     xgb_lr_pred.reshape(-1,1)))
y_train = pickle.load(open("./data/y_train.pkl","rb")).astype('int32')
X_train.shape,y_train.shape

Wall time: 103 ms


### 2.融合训练

In [4]:
%%time
from sklearn.linear_model import LogisticRegressionCV
lrcv = LogisticRegressionCV(n_jobs=4,cv=5,random_state=6)
lrcv.fit(X_train,y_train)

Wall time: 1min 31s


In [6]:
y_proba_cv = lrcv.predict_proba(X_train)
auc_cv = roc_auc_score(y_train,y_proba_cv[:,1])
print(auc_cv)

0.794636380238451


In [7]:
lrcv.coef_

array([[0.28491475, 0.44399981, 0.32817089]])

### 3.模型融合

In [8]:
blend_w = dict(rf=0.25,lgbm=0.4,xgb_lr=0.35)
blend_pred = rf_pred*blend_w['rf']+lgbm_pred*blend_w['lgbm']+\
       xgb_lr_pred*blend_w['xgb_lr']
blend_auc = roc_auc_score(y_train,blend_pred)
print(blend_auc)

0.79333644284456
